## 1-3. Setup and Exploration

In [29]:
# Importing libraries
import pandas as pd
import numpy as np

# Loading in data
df = pd.read_csv('data/Colorado River Basin Water Conflict Table.csv')

In [3]:
# Adjusting settings to see all of the columns, and then viewing first 3 rows
pd.set_option("display.max.columns", None)
df.head(3)

,Event,Search Source,Newspaper,Article Title,Duplicate,Report Date,Report Year,Event Date,Event Day,Event Month,Event Year,Conflict Present,Crisis Present,Basin,HUC6,HUC2,Place,County,County FIPS,State,State FIPS,Urban or Rural,Issue Type,Event Summary,Stakeholders,Intensity Value,Comments,Related Observation Themes,Article Text Search - water quality,Article Text Search - invasive species,Article Text Search - conservation,Article Text Search - drought,Article Text Search - flood,Article Text Search - ground water depletion,Article Text Search - depletion,Article Text Search - infrastructure,Article Text Search - fish passage,Article Text Search - instream water rights,Article Text Search - water rights,Article Text Search - intergovernmental,Article Text Search - water transfers,Article Text Search - navigation,Article Text Search - fish,Article Text Search - invasive,Article Text Search - diversion,Article Text Search - water diversion,Article Text Search - instream,Article Text Search - aquatic
0,1,USGS1-50.docx,The Durango Herald (Colorado),Tribes assert water rights on Colorado River B...,False,7-Apr-22,2022.0,NaN,NaN,4.0,2022.0,Y,N,Upper San Juan,140801,14,"Durango, CO",La Plata,8067.0,CO,8,Both,Water rights more generally,Ute Mountain and Southern Ute representatives ...,"Tribal Nations, State Government, Federal Gove...",2.0,The article highlights calls for negotiation b...,Lack of tribal representation,0,0,3,7,0,0,0,1,0,0,17,0,0,0,0,0,0,0,0,0
1,2,USGS1-50.docx,"Journal, The (Cortez, Dolores, Mancos, CO)",Native American tribes assert water rights on ...,False,7-Apr-22,2022.0,NaN,NaN,4.0,2022.0,Y,N,Upper San Juan,140801,14,"Durango, CO",La Plata,8067.0,CO,8,Both,Water rights more generally,Ute Mountain and Southern Ute representatives ...,"Southern Ute Indian Tribe, Ute Mountain Tribe,...",2.0,The article highlights calls for negotiation b...,Lack of tribal representation,0,0,2,7,0,0,0,1,0,0,17,0,0,0,0,0,0,0,0,0
2,3,USGS1-50.docx,The Salt Lake Tribune,'Very positive change.' New Utah law will be a...,False,17-Mar-22,2022.0,NaN,NaN,3.0,2022.0,N,Y,Great Salt Lake,160203,16,Great Salt Lake,NaN,NaN,UT,49,Both,Instream water rights,A bill is proposed in Utah that would expand t...,"State Government, Any Water Rights Holder, Agr...",3.0,The event is the proposal of the bill at the s...,Dishonoring the absent,0,0,1,2,0,0,0,0,0,0,12,0,0,0,1,0,0,0,12,1


In [28]:
# Data exploration
print(df['Basin'].unique())
print(df['Basin'].nunique())

['Upper San Juan' 'Great Salt Lake' 'Lower Colorado'
 'Entire Lower Colorado Basin' 'Entire Colorado River Basin' 'Lower Green'
 'South Platte' 'Lower Colorado-Lake Mead' 'Entire Upper Colorado Basin'
 'Middle Gila' 'Upper Colorado-Dirty Devil' 'Colorado Headwaters' nan
 'Little Colorado' 'White-Yampa' 'Lower Gila-Agua Fria' 'Upper Green'
 'Santa Cruz' 'Salt' 'Salton Sea'
 'Upper San Juan, Salton Sea, Salt, Rio De La Concepcion, Lower San Juan, Upper Colorado-Dirty Devil, Bill Williams, Middle Gila, Santa Cruz, Verde, Lower Gila, San Pedro-Willcox, Rio Sonoyta, Lower Colorado, Little Colorado, Lower Colorado-Lake Mead, Upper Gila, Rio De Bavispe, Lower Gila-Agua Fria '
 'Upper Colorado-Dirty Devil, Upper San Juan'
 'Upper Colorado-Dirty Devil, Lower San Juan, Little Colorado, Lower Colorado-Lake Mead'
 'Salt, Lower Colorado' 'Upper Gila' 'Lower San Juan']
25


In [26]:
# More data exploration
print(df.shape)
# print(df.dtypes)

(268, 48)


In [27]:
# More exploration
# df.info() // commented out bc I don't like how ugly it looks :(

In [27]:
df.isna().sum() # proper way to find number of NA values, because adds up 0s and 1s that represent Booleans
# df.isna().count() just counts up the number of obsverations, not considering NAs

## 5. String accessor for `pandas.Series`

In [30]:
s = pd.Series(['California; Nevada', 'Arizona', np.nan, 'Nevada; Utah'])
s

0    California; Nevada
1               Arizona
2                   NaN
3          Nevada; Utah
dtype: object

In [31]:
# str accessor (doesn't do anything by itself)
s.str

In [32]:
# Use str accessor with additional methods to perform string operations
# .split splits strings by ';' and expands output into separate columns
s.str.split(';', expand=True)

,0,1
0,California,Nevada
1,Arizona,None
2,NaN,NaN
3,Nevada,Utah


In [33]:
# Use stack() method to flatten the data frame into a series
# default is to drop NAs and None from result
s.str.split(';', expand=True).stack()

0  0    California
   1        Nevada
1  0       Arizona
3  0        Nevada
   1          Utah
dtype: object

## 6. Examine state codes

Which states in the dataset are reported as having water conflicts?

In [41]:
# filtering just states that have water conflicts
conflict = df[df['Conflict Present'] == 'Y']

# checking unique state values
conflict['State'].unique()

# oh no, there are observations with multiple states!

array(['CO', nan, 'AZ', 'OH; UT', 'UT', 'CA', 'AZ; NV', 'CO; UT; WY; NM',
       'AZ; CA', 'AZ; UT', 'NV; AZ', 'AZ; CA; CO; NV; NM; UT; WY', 'NV',
       'NM', 'UT; CO; WY', 'AZ; NM', 'WY; UT; CO', 'CO; AZ'], dtype=object)

The challenge to finding unique states in this way is that the 'State' column contains combinations of states (for example: 'WY;UT'). 

We have to split up these entries first, before we proceed.

## 8. Exploratory wrangling
## 9. Find unique state codes

In [49]:
# this creates some repetitions (for example, multiple CO!)
conflict['State'].str.split(';', expand = True).stack() # second index column is in relation to our stacked table 

# (row 0, column 2) refers to the second state listed in the first observation

conflict['State'].str.split(';', expand = True).stack().unique()

array(['CO', 'AZ', 'OH', ' UT', 'UT', 'CA', ' NV', ' WY', ' NM', ' CA',
       'NV', ' AZ', ' CO', 'NM', 'WY'], dtype=object)

In [50]:
# fix by adding a ' ' after ;
conflict['State'].str.split('; ', expand = True).stack().unique()
# can also do str.strip

# all good :)

array(['CO', 'AZ', 'OH', 'UT', 'CA', 'NV', 'WY', 'NM'], dtype=object)

#### **Bonus:** How many articles mention each state?

Different between `.count()` and `.value_counts()`:

`.count()`: returns the number of observations, in total

`.value_counts()`: returns the number of different unique values within the column

In [76]:
# Selecting state column out of original data frame
# Applying str.split() to expand the state column by unique state
# Using value_counts() to find the number

df['State'].str.split('; ', expand = True).stack().value_counts()

AZ    87
CO    45
UT    40
NV    19
CA    16
NM    13
WY     8
OH     1
TX     1
Name: count, dtype: int64

In [85]:
# what happens if I want to split a string but retain the whole data frame around it?
new_df = df
new_df['State'] = new_df['State'].str.split('; ', expand = True)
new_df

ValueError: Columns must be same length as key

In [90]:
new_df['State'].str.split('; ', expand = True)

,0,1,2,3,4,5,6
0,CO,None,None,None,None,None,None
1,CO,None,None,None,None,None,None
2,UT,None,None,None,None,None,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
263,CO,None,None,None,None,None,None
264,CO,None,None,None,None,None,None
265,AZ,CA,None,None,None,None,None
266,AZ,None,None,None,None,None,None


### How to find Python documentation

In [81]:
len?

Signature: len(obj, /)
Docstring: Return the number of items in a container.
Type:      builtin_function_or_method

In [78]:
help(str.split)

Help on method_descriptor:

split(self, /, sep=None, maxsplit=-1)
    Return a list of the substrings in the string, using sep as the separator string.
    
      sep
        The separator used to split the string.
    
        When set to None (the default value), will split on any whitespace
        character (including \\n \\r \\t \\f and spaces) and will discard
        empty strings from the result.
      maxsplit
        Maximum number of splits (starting from the left).
        -1 (the default value) means no limit.
    
    Note, str.split() is mainly useful for data that has been intentionally
    delimited.  With natural text that includes punctuation, consider using
    the regular expression module.



In [79]:
help(pd.Series.value_counts)

Help on function value_counts in module pandas.core.base:

value_counts(self, normalize: 'bool' = False, sort: 'bool' = True, ascending: 'bool' = False, bins=None, dropna: 'bool' = True) -> 'Series'
    Return a Series containing counts of unique values.
    
    The resulting object will be in descending order so that the
    first element is the most frequently-occurring element.
    Excludes NA values by default.
    
    Parameters
    ----------
    normalize : bool, default False
        If True then the object returned will contain the relative
        frequencies of the unique values.
    sort : bool, default True
        Sort by frequencies when True. Preserve the order of the data when False.
    ascending : bool, default False
        Sort in ascending order.
    bins : int, optional
        Rather than count values, group them into half-open bins,
        a convenience for ``pd.cut``, only works with numeric data.
    dropna : bool, default True
        Don't include counts